In [1]:
import sys
import tensorflow as tf
from tensorflow.keras.models import load_model
from PyQt5.QtCore import Qt
from PyQt5.QtWidgets import QApplication, QMainWindow, QPushButton, QLabel, QVBoxLayout, QWidget, QFileDialog
from PyQt5.QtGui import QPixmap
from PIL import Image
import numpy as np

# Load the trained model
model = load_model('fracture_detection_model.keras')

# Parameters for image resizing
IMG_HEIGHT = 180
IMG_WIDTH = 180

class FractureDetectionApp(QMainWindow):
    def __init__(self):
        super().__init__()

        self.setWindowTitle('Bone Fracture Detection')
        self.setGeometry(100, 100, 800, 600)  # Increase window size
        self.setStyleSheet("background-color: #f4f4f9;")  # Background color

        # Store the uploaded file path
        self.image_path = None

        # Set up UI components
        self.label = QLabel(self)
        self.label.setAlignment(Qt.AlignCenter)
        self.label.setText("Upload an image to detect fracture status.")
        self.label.setStyleSheet("font-size: 18px; color: #333; padding: 20px;")

        self.upload_button = QPushButton("Upload Image", self)
        self.upload_button.setStyleSheet("""
            background-color: #4CAF50;
            color: white;
            font-size: 16px;
            padding: 10px;
            border-radius: 5px;
        """)
        self.upload_button.setFixedSize(200, 50)
        self.upload_button.clicked.connect(self.upload_image)

        self.test_button = QPushButton("Test", self)
        self.test_button.setStyleSheet("""
            background-color: #008CBA;
            color: white;
            font-size: 16px;
            padding: 10px;
            border-radius: 5px;
        """)
        self.test_button.setFixedSize(200, 50)
        self.test_button.setEnabled(False)  # Disabled until an image is uploaded
        self.test_button.clicked.connect(self.make_prediction)

        self.result_label = QLabel(self)
        self.result_label.setAlignment(Qt.AlignCenter)
        self.result_label.setStyleSheet("font-size: 20px; color: #333; margin-top: 20px; padding: 20px;")

        # Set up layout
        layout = QVBoxLayout()
        layout.addWidget(self.label)
        layout.addWidget(self.upload_button, alignment=Qt.AlignCenter)  # Center the upload button
        layout.addWidget(self.test_button, alignment=Qt.AlignCenter)  # Center the test button
        layout.addWidget(self.result_label)

        container = QWidget()
        container.setLayout(layout)
        self.setCentralWidget(container)

    def upload_image(self):
        # Open file dialog to select image
        file_name, _ = QFileDialog.getOpenFileName(self, "Open Image", "", "Image Files (*.png *.jpg *.bmp *.jpeg)")

        if file_name:
            self.image_path = file_name  # Store the file path
            self.display_image(file_name)
            self.test_button.setEnabled(True)  # Enable the test button after image is uploaded

    def display_image(self, file_name):
        # Display the uploaded image
        pixmap = QPixmap(file_name)
        pixmap = pixmap.scaled(IMG_WIDTH, IMG_HEIGHT, Qt.KeepAspectRatio)
        self.label.setPixmap(pixmap)

    def preprocess_image(self, file_name):
        # Preprocess the image as required by the model
        img = Image.open(file_name).convert('RGB')
        img = img.resize((IMG_WIDTH, IMG_HEIGHT))
        img_array = np.array(img)
        img_array = img_array / 255.0
        img_array = np.expand_dims(img_array, axis=0)
        return img_array

    def make_prediction(self):
        if not self.image_path:
            return  # No image uploaded

        # Preprocess the image and make the prediction
        img_array = self.preprocess_image(self.image_path)
        prediction = model.predict(img_array)
        class_names = ['Fractured', 'Not Fractured']
        predicted_class = (prediction > 0.5).astype("int32")

        # Display the result
        self.result_label.setText(f"Prediction: {class_names[predicted_class[0][0]]}")
        if predicted_class[0][0] == 0:
            self.result_label.setStyleSheet("font-size: 20px; color: red;")
        else:
            self.result_label.setStyleSheet("font-size: 20px; color: green;")

# Running the app
if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = FractureDetectionApp()
    window.show()
    sys.exit(app.exec_())


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 532ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step


SystemExit: 0

c:\Users\AjayPatil\AppData\Local\Programs\Python\Python312\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
